### 학습 목표
- 부스팅 모델 중 가장 유명한 XGBoost를 활용하여 커플 성사를 예측, 그리드 서치(Grid Search)로 하이퍼파라미터를 튜닝하여 더 나은 모델을 만드는 방법을 학습

### 학습 순서
- 문제 정의
- 라이브러리 및 데이터 불러오기
- 전처리
  - 데이터 클리닝
  - 피처 엔지니어링
- 모델링 및 예측하기
- 이해하기 : 경사하강법
- 하이퍼파라미터 튜닝 : 그리드 서치
- 중요 변수 확인
- 이해하기 : XGBoost

### XGBoost 소개
- 랜덤 포레스트는 각 트리를 독립적으로 만드는 알고리즘. 반면 부스팅은 순차적으로 트리를 만들어 이전 트리로부터 더 나은 트리를 만드는 알고리즘
- 부스팅 알고리즘은 트리 모델을 기반으로 한 최신 알고리즘 중 하나로, 랜덤 포레스트보다 훨씬 더 빠른 속도와 더 좋은 예측 능력을 보여준다.
  - XGBoost, LightGBM, CatBoost 등
  - 그 중 XGBoost(eXtra Gradient Boost)가 가장 먼저 개발되기도 했고, 가장 널리 활용된다
  - XGBoost는 손실 함수뿐만 아니라 모형 복잡도까지 고려
- 부스팅 알고리즘 : 부스팅은 랜덤 포레스트에서 그 다음 세대로 진화하게 되는 중요한 개념이다. 랜덤 포레스트에서는 각각의 트리를 독립적으로, 즉 서로 관련 없이 만드는 반면, 부스팅 알고리즘에서는 트리를 순차적으로 만들면서 이전 트리에서 학습한 내용이 다음 트리를 만들 때 반영된다.

### 장점
- 예측 속도가 상당히 빠르며, 예측력 또한 좋다
- 변수 종류가 많고 데이터가 클수록 상대적으로 뛰어난 성능을 보여준다.

### 단점
- 복잡한 모델인 만큼, 해석에 어려움이 있다
- 더 나은 성능을 위한 하이퍼파라미터 튜닝이 까다롭다

### 유용한 곳
- 종속변수가 연속형 데이터인 경우든 범주형 데이터인 경우든 모두 사용 가능
- 이미지나 자연어가 아닌 표로 정리된 데이터의 경우, 거의 모든 상황에 활용 가능

## 10.1 문제 정의
- 알고리즘 : XGBoost
- 종속변수 : match(커플 성사 여부)
- 독립변수 : 상대방과 내 정보, 개인의 취향, 상대방에 대한 평가
- 문제 유형 : 분류
- 평가지표 : 정확도, 혼돈 행렬, 분류 리포트
- 사용한 모델 : XGBClassifier

## 10.2 라이브러리 및 데이터 불러오기, 데이터 확인하기

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_url = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/dating.csv'
data = pd.read_csv(file_url)

In [25]:
data.head()

,has_null,gender,age,age_o,race,race_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sincere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,like,guess_prob_liked,met,match
0,0,female,21.0,27.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,15.0,20.0,20.0,15.0,15.0,15.0,6.0,9.0,7.0,7.0,6.0,5.0,0.14,3.0,2.0,7.0,6.0,0.0,0
1,0,female,21.0,22.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,8.0,7.0,8.0,5.0,6.0,0.54,3.0,2.0,7.0,5.0,1.0,0
2,1,female,21.0,22.0,Asian/PacificIslander/Asian-American,Asian/PacificIslander/Asian-American,2.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,8.0,9.0,8.0,5.0,7.0,0.16,3.0,2.0,7.0,NaN,1.0,1
3,0,female,21.0,23.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,9.0,8.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,6.0,8.0,7.0,6.0,8.0,0.61,3.0,2.0,7.0,6.0,0.0,1
4,0,female,21.0,24.0,Asian/PacificIslander/Asian-American,Latino/HispanicAmerican,2.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,9.0,7.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,6.0,7.0,7.0,6.0,6.0,0.21,3.0,2.0,6.0,6.0,0.0,1


In [26]:
pd.options.display.max_columns = 40 

In [27]:
data.head()

,has_null,gender,age,age_o,race,race_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sincere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,like,guess_prob_liked,met,match
0,0,female,21.0,27.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,15.0,20.0,20.0,15.0,15.0,15.0,6.0,9.0,7.0,7.0,6.0,5.0,0.14,3.0,2.0,7.0,6.0,0.0,0
1,0,female,21.0,22.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,8.0,7.0,8.0,5.0,6.0,0.54,3.0,2.0,7.0,5.0,1.0,0
2,1,female,21.0,22.0,Asian/PacificIslander/Asian-American,Asian/PacificIslander/Asian-American,2.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,8.0,9.0,8.0,5.0,7.0,0.16,3.0,2.0,7.0,NaN,1.0,1
3,0,female,21.0,23.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,9.0,8.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,6.0,8.0,7.0,6.0,8.0,0.61,3.0,2.0,7.0,6.0,0.0,1
4,0,female,21.0,24.0,Asian/PacificIslander/Asian-American,Latino/HispanicAmerican,2.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,9.0,7.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,6.0,7.0,7.0,6.0,6.0,0.21,3.0,2.0,6.0,6.0,0.0,1


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 39 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   has_null                       8378 non-null   int64  
 1   gender                         8378 non-null   object 
 2   age                            8283 non-null   float64
 3   age_o                          8274 non-null   float64
 4   race                           8315 non-null   object 
 5   race_o                         8305 non-null   object 
 6   importance_same_race           8299 non-null   float64
 7   importance_same_religion       8299 non-null   float64
 8   pref_o_attractive              8289 non-null   float64
 9   pref_o_sincere                 8289 non-null   float64
 10  pref_o_intelligence            8289 non-null   float64
 11  pref_o_funny                   8280 non-null   float64
 12  pref_o_ambitious               8271 non-null   f

In [29]:
round(data.describe(), 2)

# 최댓값
# => 본인 및 상대방을 평가하는 변수(o, partner)에서는 대체로 최댓값이 10으로 일정,
# => 중요도와 관련된 변수(important)에서는 최댓값이 100부터 60,50 등 변수별로 다양
# => 이럴 때는 데이터가 어떠한 형식으로 수집되었는지 알아야 향후 데이터 클리닝 및 피처 엔지니어링을 하는 데 도움이 된다.



,has_null,age,age_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sincere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,like,guess_prob_liked,met,match
count,8378.00,8283.00,8274.00,8299.00,8299.00,8289.00,8289.00,8289.00,8280.00,8271.00,8249.00,8166.00,8091.00,8072.00,8018.00,7656.00,7302.00,8299.00,8299.00,8299.00,8289.00,8279.00,8257.00,8176.00,8101.00,8082.00,8028.00,7666.00,7311.00,8220.00,8277.00,1800.00,8138.00,8069.00,8003.00,8378.00
mean,0.87,26.36,26.36,3.78,3.65,22.50,17.40,20.27,17.46,10.69,11.85,6.19,7.18,7.37,6.40,6.78,5.47,22.51,17.40,20.27,17.46,10.68,11.85,6.19,7.18,7.37,6.40,6.78,5.47,0.20,5.53,5.57,6.13,5.21,0.05,0.16
std,0.33,3.57,3.56,2.85,2.81,12.57,7.04,6.78,6.09,6.13,6.36,1.95,1.74,1.55,1.95,1.79,2.16,12.59,7.05,6.78,6.09,6.12,6.36,1.95,1.74,1.55,1.95,1.79,2.16,0.30,1.73,4.76,1.84,2.13,0.28,0.37
min,0.00,18.00,18.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.83,1.00,0.00,0.00,0.00,0.00,0.00
25%,1.00,24.00,24.00,1.00,1.00,15.00,15.00,17.39,15.00,5.00,9.52,5.00,6.00,6.00,5.00,6.00,4.00,15.00,15.00,17.39,15.00,5.00,9.52,5.00,6.00,6.00,5.00,6.00,4.00,-0.02,5.00,2.00,5.00,4.00,0.00,0.00
50%,1.00,26.00,26.00,3.00,3.00,20.00,18.37,20.00,18.00,10.00,10.64,6.00,7.00,7.00,7.00,7.00,6.00,20.00,18.18,20.00,18.00,10.00,10.64,6.00,7.00,7.00,7.00,7.00,6.00,0.21,6.00,4.00,6.00,5.00,0.00,0.00
75%,1.00,28.00,28.00,6.00,6.00,25.00,20.00,23.81,20.00,15.00,16.00,8.00,8.00,8.00,8.00,8.00,7.00,25.00,20.00,23.81,20.00,15.00,16.00,8.00,8.00,8.00,8.00,8.00,7.00,0.43,7.00,8.00,7.00,7.00,0.00,0.00
max,1.00,55.00,55.00,10.00,10.00,100.00,60.00,50.00,50.00,53.00,30.00,10.50,10.00,10.00,11.00,10.00,10.00,100.00,60.00,50.00,50.00,53.00,30.00,10.00,10.00,10.00,10.00,10.00,10.00,0.91,10.00,20.00,10.00,10.00,8.00,1.00


## 10.3 전처리: 결측치 처리

In [30]:
data.isna().mean()

has_null                         0.000000
gender                           0.000000
age                              0.011339
age_o                            0.012413
race                             0.007520
race_o                           0.008713
importance_same_race             0.009429
importance_same_religion         0.009429
pref_o_attractive                0.010623
pref_o_sincere                   0.010623
pref_o_intelligence              0.010623
pref_o_funny                     0.011697
pref_o_ambitious                 0.012772
pref_o_shared_interests          0.015397
attractive_o                     0.025304
sincere_o                        0.034256
intelligence_o                   0.036524
funny_o                          0.042970
ambitous_o                       0.086178
shared_interests_o               0.128432
attractive_important             0.009429
sincere_important                0.009429
intellicence_important           0.009429
funny_important                  0

In [31]:
# 중요도와 관련된 변수들은 결측치 제거
data = data.dropna(subset=['pref_o_attractive', 'pref_o_sincere', 'pref_o_intelligence', 'pref_o_funny', 'pref_o_ambitious', 'pref_o_shared_interests','attractive_important', 'sincere_important', 'intellicence_important', 'funny_important', 'ambtition_important', 'shared_interests_important'])

- 기본적으로 트리 베이스 모델이라서 결측치를 채우기는 까다롭지 않다.
- 데이터에 등장하지 않을 법한 임의의 숫자, 예를 들어 -99와 같은 숫자를 채워넣는 것으로 해당 사람은 해당 항목에 "응답하지 않음"을 나타내보겠다.
- 단, 중요도와 관련된 변수들은 결측치를 제거하는 방향으로 처리
  - 곧 진행할 피처 엔지니어링에서 중요도x점수 로 계산 할 것
- 평가 변수에 관한 변수는 무응답(결측치)을 하나의 응답 종류로 간주하여 사용하기로 할 것

In [32]:
data = data.fillna(-99) # 남은 결측치는 -99로 대체

# => 가령 -99로 결측치를 대체한다면, 선형모델에서는 해당 숫자가 아웃라이어로써 작용하겠지만 트리 모델에서는 결측치라는 사실 자체가 유의미한 차이를 보인다면 -99를 분류하는 노드가 생겨날 것

## 10.4 전처리: 피처 엔지니어링

In [33]:
# 나이와 관련된 변수
# - 고려해야 할 사항 -> 결측치
# - 결측치를 -99로 채워넣었으므로 단순히 나이차를 계산해서는 안 된다.
# - '알 수 없음'의 의미로 -99를 사용
# - 또 하나는 성별과 관련된 요인. 단순한 나이 차이보다는 남자가 여자보다 많은지, 반대 경우인지도 고려

def age_gap(x):
    if x['age'] == -99: # 알수없음
        return -99
    elif x['age_o'] == -99: # 알수없음
        return -99
    elif x['gender'] == 'female': 
        return x['age_o'] - x['age']
    else:
        return x['age'] - x['age_o']

# 남녀 중 한 명이라도 나이가 -99이면 -99를 반환한다.
# 그렇지 않으면 남자가 연상이면 플러스값이, 여자가 연상이면 마이너스값이 반환된다

In [34]:
data['age_gap'] = data.apply(age_gap, axis=1)

In [35]:
data['age_gap_abs'] = abs(data['age_gap']) # 절댓값 적용

In [36]:
# 인종 데이터 관련 피처 엔지니어링
# 본인과 상대방의 인종이 같으면 1, 다르면 -1
# => 만약, 1과 0을 사용하면 밑의 importance_same_race와 곱하는 과정에서 인종 여부가 전혀 중요하지 않은 사람은 무조건 0이 나오게 되는데,
# => 이것은 적절한 변별력을 갖기 어렵다,
# 결측치는 -99를 반환

def same_race(x):
    if x['race'] == -99:
        return -99
    elif x['race_o'] == -99:
        return -99
    elif x['race'] == x['race_o']:
        return 1
    else:
        return -1     

In [37]:
data['same_race'] = data.apply(same_race, axis=1)

In [38]:
def same_race_point(x):
    if x['same_race'] == -99:
        return -99
    else:
        return x['same_race'] * x['importance_same_race']

In [39]:
data['same_race_point'] = data.apply(same_race_point, axis=1)

In [40]:
# attractive, sincere 등에 대한 평가/중요도 변수
# 평가 점수 x 중요도

# 매개변수가 3개
# => 데이터프레임, 중요도, 평가 변수

def rating(data, importance, score):
    if data[importance] == -99:
        return -99
    elif data[score] == -99:
        return -99
    else:
        return data[importance] * data[score]

In [41]:
partner_imp = data.columns[8:14]      # 상대방의 중요도
partner_rate_me = data.columns[14:20] # 본인에 대한 상대방의 평가
my_imp = data.columns[20:26]          # 본인의 중요도
my_rate_partner = data.columns[26:32] # 상대방에 대한 본인의 평가

In [42]:
# 상대방 관련 새 변수 이름을 저장하는 리스트
new_label_partner = ['attractive_p', 'sincere_partner_p', 'intelligence_p', 'funny_p', 'ambition_p', 'shared_interests_p']

# 본인 관련 새 변수 이름을 저장하는 리스트
new_label_me = ['attractive_m', 'sincere_partner_m', 'intelligence_m', 'funny_m', 'ambition_m', 'shared_interests_m']

In [43]:
for i,j,k in zip(new_label_partner, partner_imp, partner_rate_me):
    print(i,' & ',j,' & ',k)

attractive_p  &  pref_o_attractive  &  attractive_o
sincere_partner_p  &  pref_o_sincere  &  sincere_o
intelligence_p  &  pref_o_intelligence  &  intelligence_o
funny_p  &  pref_o_funny  &  funny_o
ambition_p  &  pref_o_ambitious  &  ambitous_o
shared_interests_p  &  pref_o_shared_interests  &  shared_interests_o


In [46]:
# data전체에 대해 apply를 활용하여 rating 함수를 사용
# rating 함수의 매개변수
# => x = 데이터 프레임 / j = 중요도 변수 이름 / k = 평가 변수
# => 계산 결과는 중요도 x 평가 변수인 data[i]에 저장

# 여기에서, apply안에 lambda x를 사용하면 해당 변수 i의 한 줄 한 줄의 데이터가 x로 받아져서 rating()함수로 사용된다
for i,j,k in zip(new_label_partner, partner_imp, partner_rate_me):
    data[i] = data.apply(lambda x: rating(x, j, k), axis=1) # ❶  

In [48]:
for i,j,k in zip(new_label_me, my_imp, my_rate_partner):
    data[i] = data.apply(lambda x: rating(x, j, k), axis=1)

In [47]:
data

,has_null,gender,age,age_o,race,race_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sincere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,...,like,guess_prob_liked,met,match,age_gap,age_gap_abs,same_race,same_race_point,attractive_p,sincere_partner_p,intelligence_p,funny_p,ambition_p,shared_interests_p,attractive_m,sincere_partner_m,intelligence_m,funny_m,ambition_m,shared_interests_m
0,0,female,21.0,27.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,...,7.0,6.0,0.0,0,6.0,6.0,-1,-2.0,210.0,160.0,160.0,160.0,0.0,30.0,90.0,180.0,140.0,105.0,90.0,75.0
1,0,female,21.0,22.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,...,7.0,5.0,1.0,0,1.0,1.0,-1,-2.0,420.0,0.0,0.0,280.0,0.0,0.0,105.0,160.0,140.0,120.0,75.0,90.0
2,1,female,21.0,22.0,Asian/PacificIslander/Asian-American,Asian/PacificIslander/Asian-American,2.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,...,7.0,-99.0,1.0,1,1.0,1.0,1,2.0,190.0,180.0,190.0,180.0,140.0,120.0,75.0,160.0,180.0,120.0,75.0,105.0
3,0,female,21.0,23.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,9.0,8.0,...,7.0,6.0,0.0,1,2.0,2.0,-1,-2.0,210.0,40.0,135.0,320.0,45.0,40.0,105.0,120.0,160.0,105.0,90.0,120.0
4,0,female,21.0,24.0,Asian/PacificIslander/Asian-American,Latino/HispanicAmerican,2.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,9.0,7.0,...,6.0,6.0,0.0,1,3.0,3.0,-1,-2.0,240.0,70.0,180.0,60.0,90.0,140.0,75.0,120.0,140.0,105.0,90.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8372,1,male,25.0,24.0,European/Caucasian-American,European/Caucasian-American,1.0,1.0,10.0,15.0,30.0,20.0,15.0,10.0,8.0,8.0,7.0,7.0,8.0,6.0,...,4.0,4.0,0.0,0,1.0,1.0,1,1.0,80.0,120.0,210.0,140.0,120.0,60.0,490.0,0.0,75.0,75.0,0.0,-99.0
8373,1,male,25.0,26.0,European/Caucasian-American,Latino/HispanicAmerican,1.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,10.0,5.0,3.0,2.0,6.0,5.0,...,2.0,5.0,0.0,0,-1.0,1.0,-1,-1.0,100.0,50.0,90.0,40.0,60.0,75.0,210.0,0.0,75.0,75.0,-99.0,-99.0
8374,1,male,25.0,24.0,European/Caucasian-American,Other,1.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,6.0,3.0,7.0,3.0,7.0,2.0,...,4.0,4.0,0.0,0,1.0,1.0,-1,-1.0,300.0,60.0,70.0,15.0,70.0,10.0,280.0,0.0,120.0,60.0,0.0,-99.0
8376,1,male,25.0,22.0,European/Caucasian-American,Asian/PacificIslander/Asian-American,1.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,5.0,7.0,5.0,5.0,3.0,6.0,...,5.0,5.0,0.0,0,3.0,3.0,-1,-1.0,50.0,175.0,125.0,50.0,30.0,120.0,280.0,0.0,75.0,60.0,-99.0,0.0


In [49]:
# 모델링에 앞서 마지막으로 object 타입 변수들은 문자 형태이기 때문에,
# 숫자 형태가 되게끔 더미 변수로 바꿔준다.
# gender, race, race_o 단 3개 변수만이 object 타입

# 더미 변수로 변환
data = pd.get_dummies(data, columns=['gender','race','race_o'], drop_first=True)

## 10.5 모델링 및 평가

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('match',axis=1), data['match'], test_size=0.2, random_state=100)

In [54]:
import xgboost as xgb

XGBoost는 크게 3가지 학습 방법 제공
- 랜덤 포레스트와 마찬가지로 분류와 회귀의 fit()함수를 제공하고, 추가로 train함수도 제공
- 이 장에서는 분류 함수를 사용하여 모델링

In [56]:
# 총 3가지 하이퍼파라미터 임의 값 설정
model = xgb.XGBClassifier(n_estimators = 500, max_depth = 5, random_state=100) # 모델 객체 생성


In [57]:
model.fit(X_train, y_train) # 훈련

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=500,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=100, ...)

In [58]:
pred = model.predict(X_test) # 예측

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(y_test, pred) # 정확도 평가

0.8616236162361623

In [59]:
print(confusion_matrix(y_test, pred)) # 혼돈 행렬 출력

[[1291   74]
 [ 151  110]]


In [60]:
print(classification_report(y_test, pred))

# 차례로 정밀도, 재현율, F1-점수, 인덱스에 해당하는 점수

              precision    recall  f1-score   support

           0       0.90      0.95      0.92      1365
           1       0.60      0.42      0.49       261

    accuracy                           0.86      1626
   macro avg       0.75      0.68      0.71      1626
weighted avg       0.85      0.86      0.85      1626



- 기본적으로 classification_report() 결과물은 종속변수의 값인 0과 1 각각에 대하여 한 줄씩 나타난다.
- 대부분은 예측하려는 경우를 1로 두기 때문에 1에 대한 값들을 주로 해석, 0에 대한 값들은 필요에 따라 확인
- 이 데이터의 목푯값 중 0의 비율이 87%이기 때문에 정밀도(precision), 재현율(recall), F1-점수(f1-score) 모두 0.9 이상의 높은 값을 보여주므로 중요도가 떨어진다.
- 반면, 1에 대한 값들은 0.6, 0.42, 0.49, 등으로 상대적으로 확연히 낮다

TN | FP
FN | TP

- TN : 음성을 음성으로 판단
- FP : 음성을 양성으로 판단 (1종 오류)
- FN : 양성을 음성으로 판단 (2종 오류)
- TP : 양성을 양성으로 판단

- 정밀도(precision) : 1로 예측한 경우 중, 얼마만큼이 실제로 1인지를 나타낸다.
  - TP / TP + FP 
  - FP가 커질수록 분모가 커지기 때문에, 정밀도는 낮아진다. 즉, 1종 오류와 관련
- 재현율(recall) : 실제로 1중에, 얼마만큼을 1로 예측했는지
  - TP / TP + FN
  - FN가 커질수록 분모가 커지기 때문에, recall 값이 작아진다. Type 2 Error와 관련
- F1-점수 (f1-score)는 정밀도와 재현율의 조화 평균
  - 2 x ( (precision x recall) / (precision + recall) )
  - 조화평균값이므로 정밀도와 재현율이 높을 때 함꼐 높아지며, 둘의 값이 비슷할수록 더 높은 값

### 어떤 오륫값을 살펴봐야 하는지는 분석의 목적에 따라 다르다.
- 1종 오류가 중유하면 정밀도
- 2종 오류가 중유하면 재현율
- 중요한 오류 유형이 없다면 F1-점수를 보는 게 무난한 방법

## 10.6 이해하기 : 경사하강법 (gradient descent)
- 머신러닝이 학습시킬 때 최소의 오차를 찾는 방법
- 오차 함수에 대한 경사도(미분계수를 기준으로 매개변수를 반복적으로 이동해가며 최소 오차를 찾는다
- 매개변수는 선형 회귀로 치면 계수(변수에 대한 기울기 값)에 해당한다.
- 경사 하강법은, 접선의 기울기가 0인 곳을 찾는 방법
- 정확히 최솟값을 찾아낸다기보다 미분계수가 최대한 0에 가깝도록 계속 움직여서 최솟값의 근삿값에 다다르르 방법

- 가령 에러에 대한 그래프가 4차 함수의 형식을 띄고 있다면, 경사하강법이 최솟값이라고 여길 만한 부분, 즉 미분계수가 0인 지점이 두 곳이다.
  - 노란점을 지역 최솟값
  - 빨간점을 전역 최솟값이라고 한다.
  - 이런 상황에서 학습률(learning late)이 충분히 크지 않다면, 지역 최솟값을 최솟값으로 판단할 수 있으니, 학습률을 충분한 크기로 주어 지역 최솟값을 지나 전역 최솟값으로 향할 수 있게 해야 한다.

- 수많은 변수를 가지고 있는 실제 데이터에서는 최소 오차를 찾기 매우 어렵다. 따라서 경사하강법을 사용하는 것이 효율적

## 10.7 하이퍼파라미터 튜닝 : 그리드 서치
- 그리드 서치를 활용한 하이퍼파라미터 튜닝
- 그리드 서치를 이용하면 한 번 시도로, 수백 가지 하이퍼파라미터값을 시도할 수 있다.

- 원리
  - 그리드 서치에 입력할 하이퍼파라미터 후보들을 입력하면, 각 조합에 대해 모두 모델링해보고 최적의 결과가 나오는 하이퍼파라미터 조합을 알려준다.

- 보통 그리드 서치에서는 교차검증도 함께 사용하기 때문에 교차검증의 횟수만큼 곱해진 횟수가 모델링된다.
- 가령 9가지 경우의 수에서 K-폴드값을 5로 교차검증을 한다면 45회의 모델링 진행

In [61]:
from sklearn.model_selection import GridSearchCV

In [62]:
# 그리드 서치에 넣어줄 매개변수 4개를 딕셔너리 형태로 입력
parameters = {
              'learning_rate': [0.01, 0.1, 0.3],
              'max_depth': [5,7,10],
              'subsample': [0.5,0.7,1],                 
              'n_estimators': [300, 500, 1000]
                }

- learning_rate : 경사하강법에서 '매개변수'를 얼마만큼 이동해가면서 최소 오차를 찾을지, 그 보폭의 크기를 결정하는 하이퍼파라미터
  - 기본적으로 보폭은 미분계수에 의해 결정되지만, learning_rate를 크게 하면 더 큰 보폭을, 작게 하면 그만큼 작은 보폭으로 움직인다
  - 보폭이 너무 작은 경우는, 최소 에러값을 찾는데 상당히 오래 걸리고 지역 최소해에 빠질 가능성도 높다.
  - 너무 큰 학습률은 너무 성큼 움직이기 때문에 최소 에러를 정확히 찾지 못하고 좌우로 계속 넘어다닌다.

- subsample : 모델을 학습시킬 때 일부 데이터만 사용하여 각 트리를 만든다. 0.5를 쓰면 데이터의 절반씩만 랜덤 추출하여 트리를 만든다. 이 또한 오버피팅을 방지하는 데 도움이 된다.
- n_estimators : 전체 나무의 개수를 정한다

In [63]:
model = xgb.XGBClassifier()

In [64]:
gs_model = GridSearchCV(model, parameters, n_jobs=-1, scoring='f1', cv = 5)

In [65]:
# 하이퍼파라미터셋이 총 4종류에 3개씩 값이 있으니 81번의 모델링을 진행하고 나서,
# 교차검증 5회를 실행하므 총 405번의 모델링 작업이 이뤄진다.

gs_model.fit(X_train, y_train)

In [ ]:
# 그리드 서치는 학습이 완료된 후, 가장 좋은 성능을 보인 하이퍼파라미터 조합 정보를 보관
gs_model.best_params_ # 최적의 하이퍼파라미터 출력

In [ ]:
# 또한 이전의 모델링 과정과 동일하게, 그리드 서치를 이용해서 새로운 데이터를 예측할 수도 있다.
# 이때 적용되는 모델은 최적의 하이퍼파라미터 조합이 자동으로 반영된다.
pred = gs_model.predict(X_test)

In [ ]:
accuracy_score(y_test, pred)

In [ ]:
# 정확도는 미세하고 올라갔고, F1-점수는 0.02 상승
print(classification_report(y_test, pred))

## 10.8 중요 변수 확인
- 선형 회귀와 로지스틱 회귀에서는 계수로, 결정 트리에서는 노드의 순서로 변수의 영향력을 확인
- 부스팅 모델은 이전 모델들보다 복잡한 알고리즘이기 때문에 단순히 변수의 영향력을 파악하지 어렵지만, XGBoost 에 내장된 함수는 변수의 중요도까지 계산해줌.
  - 단, 그리드 서치로 학습된 모델에서는 이 기능을 사용할 수 없으니 그리드 서치에서 찾은 최적의 하이퍼파라미터 조합으로 다시 한번 학습

In [ ]:
model = xgb.XGBClassifier(learning_rate = 0.3, max_depth = 5, n_estimators = 1000, subsample = 0.5, random_state=100)

model.fit(X_train, y_train)


In [ ]:
# 변수 중요로(피처 중요도, feature importance) 확인
model.feature_importances_

In [ ]:
feature_imp = pd.DataFrame({'features': X_train.columns, 'values': model.feature_importances_})

In [ ]:
feature_imp.head()

In [ ]:
plt.figure(figsize=(20, 10)) # 그래프 크기 설정
sns.barplot(x='values', y='features', # ❶
data=feature_imp.sort_values(by='values', ascending=False).head(10)) # ❷

## 10.9 이해하기 : XGBoost

### 트리 모델의 진화 과정부스팅 
결정트리 -> 배깅 -> 랜덤포레스트 -> 부스팅 -> 경사 부스팅 -> XG 부스팅

- 결정트리 : 가장 기본적인 트리 모델
- 랜덤 포레스트 : 데이터와 변수의 일부를 사용하여 여러 트리를 만드는 모델
- 부스팅 : 이전 트리의 학습 내용을 다음 트리에 반영하여 여러 트리를 만듦
- 경사 부스팅 : 부스팅에 경사 하강법을 적용한 모델
- XG 부스팅 : 경사 부스팅에서 계산 속도와 알고리즘 효율을 개선한 모댈
  - 2차 도함수 활용과 정규화 하이퍼파라미터 지원


### 배깅
- 부트스트랩 훈련셋을 사용하는 트리 모델
- 부트스트랩 : 데이터의 일부분을 무작위로 추출
  - 이러한 식으로 추출한 데이터의 여러 부분집합을 사용해 여러 트리를 만들어 오버피팅 방지

### 부스팅과 에이다부스트
- 에이다 부스트 : 단계적으로 트리를 만들 때 이전 단계에서의 분류 결과에 따라 각 데이터에 가중치를 부여/수정

### 경사 부스팅과 XGBoost
- 에이다부스트는 각 데이터에 가중치를 부여/수정하는 방식으로 트리를 만듦
- 반면 경사 부스팅은 경사하강법을 이용하여, 이전 모델의 에러를 기반으로 다음 트리를 만듦
  - XGBoost, LightGBM, Catboost